In [29]:
import requests
import os
import json
import pandas as pd
FOURSQUARE_API_KEY = os.environ.get("FOURSQUARE_API_KEY")
YELP_API_KEY = os.environ.get("YELP_API_KEY")

In [39]:
import import_ipynb
import city_bikes

In [43]:
df_bike.head()  # This should display the first few rows of the DataFrame


,station_name,latitude,longitude,total_bikes
0,Summerhill Ave / Maclennan Ave,43.685924,-79.376304,11
1,Queen St E / Joseph Duggan Rd,43.667763,-79.308117,18
2,Victoria Park Ave / Danforth Ave,43.691468,-79.288619,18
3,Navy Wharf Crt / Bremner Blvd,43.640722,-79.391051,7
4,420 Wellington St W,43.643834,-79.396649,14


In [50]:
df_bike.shape

(852, 4)

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [46]:
# Define Foursquare endpoint for searching places
foursquare_url = 'https://api.foursquare.com/v3/places/search'
headers = {
    "Accept": "application/json",
    "Authorization": FOURSQUARE_API_KEY
}

def foursquare_places(latitude, longitude, query, radius=1000):
    params = {
        "query": query,
        "ll": f"{latitude},{longitude}",
        "radius": radius,
        "limit": 10  # Limit to 10 results
    }
    
    response = requests.get(foursquare_url, headers=headers, params=params)
    return response.json()  # Return JSON data if the request is successful

# Using Lat Long of Toronto
latitude = 43.6532
longitude = -79.3832
restaurants = foursquare_places(latitude, longitude, "restaurant")
print(restaurants)


{'results': [{'fsq_id': '537773d1498e74a75bb75c1e', 'categories': [{'id': 13028, 'name': 'Breakfast Spot', 'short_name': 'Breakfast', 'plural_name': 'Breakfast Spots', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/breakfast_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 98, 'geocodes': {'main': {'latitude': 43.653268, 'longitude': -79.382074}, 'roof': {'latitude': 43.653268, 'longitude': -79.382074}}, 'link': '/v3/places/537773d1498e74a75bb75c1e', 'location': {'address': '483 Bay St', 'country': 'CA', 'cross_street': 'Albert Street', 'formatted_address': '483 Bay St (Albert Street), Toronto ON M5G 2C9', 'locality': 'Toronto', 'postcode': 'M5G 2C9', 'region': 'ON'}, 'name': 'Eggspectation Bell Trinity Square', 'related_places': {'parent': {'fsq_id': '4ae5df5af964a520c4a221e3', 'categories': [{'id': 11124, 'name': 'Office', 'short_name': 'Office', 'plural_name': 'Offices', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/b

In [47]:
results = restaurants.get('results', []) 
restaurants_df = pd.DataFrame(results) 
restaurants_df.head() 

,fsq_id,categories,chains,closed_bucket,distance,geocodes,link,location,name,related_places,timezone
0,537773d1498e74a75bb75c1e,"[{'id': 13028, 'name': 'Breakfast Spot', 'shor...",[],VeryLikelyOpen,98,"{'main': {'latitude': 43.653268, 'longitude': ...",/v3/places/537773d1498e74a75bb75c1e,"{'address': '483 Bay St', 'country': 'CA', 'cr...",Eggspectation Bell Trinity Square,{'parent': {'fsq_id': '4ae5df5af964a520c4a221e...,America/Toronto
1,4d306dd82748b60c62b6dba0,"[{'id': 13065, 'name': 'Restaurant', 'short_na...",[],VeryLikelyOpen,216,"{'main': {'latitude': 43.654458, 'longitude': ...",/v3/places/4d306dd82748b60c62b6dba0,"{'address': '220 Yonge St', 'address_extended'...",Trattoria Mercatto,{'parent': {'fsq_id': '4ad77a12f964a520260b21e...,America/Toronto
2,4ad4c060f964a5207cf720e3,"[{'id': 13338, 'name': 'Seafood Restaurant', '...",[],LikelyOpen,312,"{'main': {'latitude': 43.650598, 'longitude': ...",/v3/places/4ad4c060f964a5207cf720e3,"{'address': '56 Temperance St', 'country': 'CA...",John & Sons Oyster House,{},America/Toronto
3,57d0a3fb498e0fb29e892eed,"[{'id': 13343, 'name': 'South American Restaur...",[],LikelyOpen,298,"{'main': {'latitude': 43.651907, 'longitude': ...",/v3/places/57d0a3fb498e0fb29e892eed,"{'address': '176 Yonge St', 'country': 'CA', '...",Leña Restaurante,{},America/Toronto
4,5d782433f6e3190008b1a914,"[{'id': 13263, 'name': 'Japanese Restaurant', ...",[],VeryLikelyOpen,303,"{'main': {'latitude': 43.655187, 'longitude': ...",/v3/places/5d782433f6e3190008b1a914,"{'address': '157 Dundas St W', 'country': 'CA'...",Gyubee Japanese Grill,{},America/Toronto


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [56]:
# Starting with an empty list
poi_data = []

#Selecting first 5 stations from the df_bike 

selected_stations = df_bike.head(5)

# Iterate through each bike station in df_bike dataframe from city_bikes
for index, row in selected_stations.iterrows():
    latitude = row['latitude'] 
    longitude = row['longitude']
    station_name = row['station_name']
    
    # Query for restaurants, bars, and banks
    for query in ["restaurant", "bar", "bank"]:
        pois = foursquare_places(latitude, longitude, query, radius=50)
        
        if pois and 'results' in pois:
            for place in pois['results']:
                distance = place.get('distance', None)
                poi_info = {
                    'station_name': station_name,
                    'name': place.get('name', ''),
                    'latitude': latitude,
                    'longitude': longitude,
                    'category': query,
                    'distance': distance 
                }
                poi_data.append(poi_info)  # Add to the empty list

# Create a df from the data
pois_df = pd.DataFrame(poi_data)

# Check the first few rows of the new df
pois_df.head()

,station_name,name,latitude,longitude,category,distance
0,Summerhill Ave / Maclennan Ave,Dolce Bakery,43.685924,-79.376304,restaurant,38
1,Summerhill Ave / Maclennan Ave,Ashbury Cleaners,43.685924,-79.376304,bar,24
2,Summerhill Ave / Maclennan Ave,Rosedale's Finest,43.685924,-79.376304,bar,27
3,Summerhill Ave / Maclennan Ave,Rosedale 1 Hour Cleaners,43.685924,-79.376304,bar,39
4,Summerhill Ave / Maclennan Ave,Dolce Bakery,43.685924,-79.376304,bar,38


Looks like there are no rating provided with foresquare places. Will make another call and modify the code.

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating